In [1]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
from scipy import ndimage
import random
from scipy.optimize import leastsq
from scipy import ndimage
from scipy.optimize import least_squares

In [2]:
hdu = fits.open('3C264-1s-hanii-STARSUB.fits')
qso_data = hdu[1].data
qso_error = hdu[2].data
qso_header = hdu[1].header
wavestart = qso_header['CRVAL3']
wavint = qso_header['CD3_3']
wave = wavestart+np.arange(qso_data.shape[0])*wavint#. This is the same as the one below.
qso_data[np.isnan(qso_data)] = 0.0000001
qso_error[np.isnan(qso_error)] = 0.000001
(central_y,central_x) = ndimage.measurements.maximum_position(qso_data[1584,:,:])
print (central_x,central_y)

57 57


In [3]:
mini_cube = qso_data[:,central_y - 22:central_y + 23,central_x - 22:central_x + 23]
mini_cube_error = qso_error[:,central_y - 22:central_y + 23,central_x - 22:central_x + 23]
qso_header['CRPIX1'] = qso_header['CRPIX1'] - (central_x - 22)
qso_header['CRPIX2'] = qso_header['CRPIX2'] - (central_y - 22)
new_hdu = fits.HDUList([fits.PrimaryHDU(mini_cube),fits.ImageHDU(mini_cube_error)])
new_hdu[0].header = qso_header
wave = np.arange(wavestart,(wavestart+(wavint*mini_cube.shape[0])),wavint)

In [4]:
def redshift(vel):
    return vel/300000.0

def line_width(vel_sigma,rest_line,inst_res_fwhm):
    sigma = vel_sigma/(300000.0-vel_sigma)*rest_line
    return np.sqrt(sigma**2+(inst_res_fwhm/2.354)**2)

def gauss(wave,amplitude,vel,vel_sigma, rest_wave,inst_res_fwhm):
    line = (amplitude)*exp(-(wave-(rest_wave*(1+redshift(vel))))**2/(2*(line_width(vel_sigma, rest_wave,inst_res_fwhm))**2))
    return line

def Ha_gauss(wave,amp_Ha,vel,vel_sigma):
    Ha = gauss(wave,amp_Ha,vel,vel_sigma,6562.85,2.5)
    return Ha

def NII_doublet_gauss(wave,amp_N6583,vel,vel_sigma):
    N_6548 = 0.33*gauss(wave,amp_N6583,vel,vel_sigma,6548.05,2.5)
    N_6583 = gauss(wave,amp_N6583,vel,vel_sigma,6583.45,2.5)
    return N_6548+N_6583

def SII_doublet_gauss(wave,amp_S6716,amp_S6731,vel,vel_sigma):
    S_6716 = gauss(wave,amp_S6716,vel,vel_sigma,6716.44,2.5)
    S_6731 = gauss(wave,amp_S6731,vel,vel_sigma,6730.82,2.5)
    return S_6716+S_6731

def full_gauss(p,wave,data,error):   
    (amp_Ha,amp_Ha_br,amp_N6583,amp_N6583_br,amp_S6716,amp_S6716_br,amp_S6731,amp_S6731_br,vel_core,vel_core_sigma,vel_wing, vel_wing_sigma,m,c) = p
    narrow_Ha = Ha_gauss(wave,amp_Ha,vel_core,vel_core_sigma)
    broad_Ha = Ha_gauss(wave,amp_Ha_br,vel_wing,vel_wing_sigma)
    narrow_NII = NII_doublet_gauss(wave,amp_N6583,vel_core,vel_core_sigma)
    broad_NII = NII_doublet_gauss(wave,amp_N6583_br,vel_wing,vel_wing_sigma)
    narrow_SII = SII_doublet_gauss(wave,amp_S6716,amp_S6731,vel_core,vel_core_sigma)
    broad_SII = SII_doublet_gauss(wave,amp_S6716_br,amp_S6731_br,vel_wing,vel_wing_sigma)
    cont = (wave/1000.0)*m+c
    return (narrow_Ha+broad_Ha+narrow_NII+broad_NII+narrow_SII+broad_SII+cont-data)/error

In [5]:
z =0.02172
k = 1+z
c = 300000
central_vel = c*z
wave = wave
mini_data = mini_cube
mini_error = mini_cube_error

 
select = (wave > 6400*k) & (wave < 6800*k)
par = np.zeros((14,mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
err = np.zeros((14,mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
fitted = np.zeros((np.shape(wave[select])[0],mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)
residuals = np.zeros((np.shape(wave[select])[0],mini_data.shape[1],mini_data.shape[2]),dtype=np.float32)



In [6]:
for i in range(mini_data.shape[1]):
    for j in range(mini_data.shape[2]):
        x = wave[select]   
        y = mini_data[:,i,j][select]
        y_err = mini_error[:,i,j][select]
       
        lower_bounds = [0,0,0,0,0,0,0,0,central_vel - 1000,0,central_vel - 1000,0,-np.inf,-np.inf]
        upper_bounds = [np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,central_vel + 1000,300,central_vel + 1000,1000,np.inf,np.inf]
        bounds_p_init = (lower_bounds,upper_bounds)

        try:
            result = least_squares(full_gauss,x0=[300,30,100,10,10,1,10,1,6516,100,6416,200,0,2],bounds=bounds_p_init,args = (x,y,y_err),max_nfev=10000000)
            popt1 = result['x'] 

        except RuntimeError or RuntimeWarning:
            popt1 = [1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,central_vel,1e-5,central_vel,1e-5,0.0001,0.01]
        
        par[:,i,j] = popt1

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: scipy.exp is deprecated and will be removed in SciPy 2.0.0, use numpy.exp instead
  if __name__ == '__main__':


In [7]:
hdus=[]
hdus.append(fits.PrimaryHDU())
hdus.append(fits.ImageHDU(par[0,:,:],name='amp_Ha'))
hdus.append(fits.ImageHDU(par[1,:,:],name='amp_Ha_br'))
hdus.append(fits.ImageHDU(par[2,:,:],name='amp_N6583'))
hdus.append(fits.ImageHDU(par[3,:,:],name='amp_N6583_br'))
hdus.append(fits.ImageHDU(par[4,:,:],name='amp_S6716'))
hdus.append(fits.ImageHDU(par[5,:,:],name='amp_S6716_br'))
hdus.append(fits.ImageHDU(par[6,:,:],name='amp_S6731'))
hdus.append(fits.ImageHDU(par[7,:,:],name='amp_S6731_br'))
hdus.append(fits.ImageHDU(par[8,:,:],name='vel_core'))
hdus.append(fits.ImageHDU(par[9,:,:],name='vel_core_sigma'))
hdus.append(fits.ImageHDU(par[10,:,:],name='vel_wing'))
hdus.append(fits.ImageHDU(par[11,:,:],name='vel_wing_sigma'))
hdus.append(fits.ImageHDU(par[12,:,:],name='m'))
hdus.append(fits.ImageHDU(par[13,:,:],name='c'))
hdu = fits.HDUList(hdus)

hdu.writeto('test.fits',overwrite=True)